<a href="https://colab.research.google.com/github/ttury/Deep-Learning-For-Natural-Language-Processing/blob/master/LSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 실습에 사용한 DTM

|-|과일이|길고|노란|먹고|바나나|사과|싶은|저는|좋아요|
|---|---|---|---|---|---|---|---|---|---|
|문서 1|0|0|0|1|0|1|1|0|0|
|문서 2|0|0|0|1|1|0|1|0|0|
|문서 3|0|1|1|0|2|0|0|0|0|
|문서 4|1|0|0|0|0|0|0|1|1|

In [2]:
import numpy as np
A = np.array([[0, 0, 0, 1, 0, 1, 1, 0, 0], [0, 0, 0, 1, 1, 0, 1, 0, 0], [0, 1, 1, 0, 2, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 1, 1]])
m, n = np.shape(A)
print(m, n)

4 9


In [44]:
# U = m x m 직교행렬, V = n x n 직교행렬, S = m x n 직사각 대각행렬
U, s, VT = np.linalg.svd(A, full_matrices = True) # full SVD

# U
print(U.round(2)) # 소수점 2번째 자리까지
print(np.shape(U))
print()

# S
print(s.round(2)) # 대각 행렬 전체가 아닌 특이값(대각선 성분, 내림차순)만 존재, 
print(np.shape(s))
print()

S = np.zeros((m, n)) # 대각 행렬 형태 생성
S[:4, :4] = np.diag(s)
# ndarray는 튜플로 슬라이싱 가능, np.diag로 대각행렬 만듬(매개변수에 따라 달라짐 벡터 -> 행렬, 행렬 -> 벡터)
print(S.round(2))
print(S.shape)
print()

# V의 전치 행렬
print(VT.round(2))
print(VT.shape)
print()

print(np.allclose(A, np.dot(np.dot(U, S), VT).round(2)))

[[-0.24  0.75  0.   -0.62]
 [-0.51  0.44 -0.    0.74]
 [-0.83 -0.49 -0.   -0.27]
 [-0.   -0.    1.    0.  ]]
(4, 4)

[2.69 2.05 1.73 0.77]
(4,)

[[2.69 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   2.05 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   1.73 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.77 0.   0.   0.   0.   0.  ]]
(4, 9)

[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]
 [ 0.58 -0.    0.    0.   -0.    0.   -0.    0.58  0.58]
 [ 0.   -0.35 -0.35  0.16  0.25 -0.8   0.16 -0.   -0.  ]
 [-0.   -0.78 -0.01 -0.2   0.4   0.4  -0.2   0.    0.  ]
 [-0.29  0.31 -0.78 -0.24  0.23  0.23  0.01  0.14  0.14]
 [-0.29 -0.1   0.26 -0.59 -0.08 -0.08  0.66  0.14  0.14]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19  0.75 -0.25]
 [-0.5  -0.06  0.15  0.24 -0.05 -0.05 -0.19 -0.25  0.75]]
(9, 9)

True


In [56]:
# Truncated SVD (절단된 SVD)

t = 2 # 토픽 개수를 정하는 하이퍼파라미터

S = S[:t, :t] # S의 경우 t x t로 자름
print(S.round(2))
print()

U = U[:, :t] # 문서의 개수 x 토픽의 개수 -> 문서의 잠재 의미 표현
print(U.round(2))
print()

VT = VT[:t, :] # 토픽의 개수 x 단어의 개수 -> 단어의 잠재 의미 표현
print(VT.round(2))
print()

A_prime = np.dot(np.dot(U, S), VT)
print(A)
print()
print(A_prime.round(2))

[[2.69 0.  ]
 [0.   2.05]]

[[-0.24  0.75]
 [-0.51  0.44]
 [-0.83 -0.49]
 [-0.   -0.  ]]

[[-0.   -0.31 -0.31 -0.28 -0.8  -0.09 -0.28 -0.   -0.  ]
 [ 0.   -0.24 -0.24  0.58 -0.26  0.37  0.58 -0.   -0.  ]]

[[0 0 0 1 0 1 1 0 0]
 [0 0 0 1 1 0 1 0 0]
 [0 1 1 0 2 0 0 0 0]
 [1 0 0 0 0 0 0 1 1]]

[[ 0.   -0.17 -0.17  1.08  0.12  0.62  1.08 -0.   -0.  ]
 [ 0.    0.2   0.2   0.91  0.86  0.45  0.91  0.    0.  ]
 [ 0.    0.93  0.93  0.03  2.05 -0.17  0.03  0.    0.  ]
 [ 0.    0.    0.    0.    0.   -0.    0.    0.    0.  ]]


In [58]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups # 20개의 다른 주제를 가진 뉴스그룹 데이터

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
len(documents)

11314

In [64]:
print(documents[1])
print()
print(dataset.target_names)








Yeah, do you expect people to read the FAQ, etc. and actually accept hard
atheism?  No, you need a little leap of faith, Jimmy.  Your logic runs out
of steam!







Jim,

Sorry I can't pity you, Jim.  And I'm sorry that you have these feelings of
denial about the faith you need to get by.  Oh well, just pretend that it will
all end happily ever after anyway.  Maybe if you start a new newsgroup,
alt.atheist.hard, you won't be bummin' so much?






Bye-Bye, Big Jim.  Don't forget your Flintstone's Chewables!  :) 
--
Bake Timmons, III

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [77]:
news_df = pd.DataFrame({'document':documents})
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ") # 알파벳 외 삭제
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))
# 길이 3 이하인 단어 삭제, (구분자).join(리스트) -> 리스트의 원소를 구분자로 구분해 합침
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [79]:
print(news_df['clean_doc'][1])

yeah expect people read actually accept hard atheism need little leap faith jimmy your logic runs steam sorry pity sorry that have these feelings denial about faith need well just pretend that will happily ever after anyway maybe start newsgroup atheist hard bummin much forget your flintstone chewables bake timmons


In [81]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [86]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x : x.split())
tokenized_doc = tokenized_doc.apply(lambda x : [item for item in x if x not in stop_words])

print(tokenized_doc[1])

['yeah', 'expect', 'people', 'read', 'actually', 'accept', 'hard', 'atheism', 'need', 'little', 'leap', 'faith', 'jimmy', 'your', 'logic', 'runs', 'steam', 'sorry', 'pity', 'sorry', 'that', 'have', 'these', 'feelings', 'denial', 'about', 'faith', 'need', 'well', 'just', 'pretend', 'that', 'will', 'happily', 'ever', 'after', 'anyway', 'maybe', 'start', 'newsgroup', 'atheist', 'hard', 'bummin', 'much', 'forget', 'your', 'flintstone', 'chewables', 'bake', 'timmons']


In [88]:
# sklearn의 TfidfVectorizer는 토큰화가 되어 있지 않은 텍스트를 사용하기 때문에 역토큰화 수행

detokenized_doc = list()
for i in range(len(news_df)):
  t = ' '.join(tokenized_doc[i])
  detokenized_doc.append(t)

print(detokenized_doc[1])

yeah expect people read actually accept hard atheism need little leap faith jimmy your logic runs steam sorry pity sorry that have these feelings denial about faith need well just pretend that will happily ever after anyway maybe start newsgroup atheist hard bummin much forget your flintstone chewables bake timmons


In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer

MAX_FEATURES = 1000 # TF-IDF 행렬을 만들 때 사용할 최대 단어 개수

vectorizer = TfidfVectorizer(stop_words='english',
                             max_features = MAX_FEATURES,
                             max_df = 0.5,
                             smooth_idf = True)

TF_IDF = vectorizer.fit_transform(news_df['clean_doc'])
print(TF_IDF.shape)

(11314, 1000)


In [96]:
from sklearn.decomposition import TruncatedSVD

svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)
svd_model.fit(TF_IDF)
print(svd_model.components_.shape) # 절단된 SVD 중 VT(토픽의 수 x 단어의 수)

(20, 1000)


In [104]:
terms = vectorizer.get_feature_names()

def get_topics(components, feature_names, n=5):
  for index, topic in enumerate(components):
    print("Topic %d:" %(index + 1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1 :-1]])
    '''
    print에서 %를 사용하면 변수 출력 가능
    argsort 함수 : numpy에서 사용가능, 오름차순 정렬했을 때의 인덱스(value 아님)의 순서 출력
    ex) 1 6 4 2 3 -> 0 3 4 2 1
    [:-n - 1 :-1]  -> argsort가 오름차순이므로 큰 값부터 n개의 값을 출력하기 위해 사용
    '''

get_topics(svd_model.components_, terms)

Topic 1: [('just', 0.20887), ('like', 0.20469), ('know', 0.19349), ('people', 0.18318), ('think', 0.1697)]
Topic 2: [('thanks', 0.32763), ('windows', 0.28786), ('card', 0.18019), ('drive', 0.16864), ('mail', 0.15261)]
Topic 3: [('game', 0.34011), ('team', 0.30311), ('year', 0.26894), ('games', 0.23784), ('drive', 0.17472)]
Topic 4: [('drive', 0.46159), ('scsi', 0.17188), ('disk', 0.14451), ('hard', 0.13805), ('problem', 0.12763)]
Topic 5: [('drive', 0.39993), ('know', 0.28768), ('thanks', 0.24917), ('does', 0.24678), ('just', 0.17387)]
Topic 6: [('just', 0.55559), ('like', 0.23559), ('windows', 0.23078), ('know', 0.15795), ('does', 0.11156)]
Topic 7: [('just', 0.43264), ('like', 0.22858), ('mail', 0.15052), ('bike', 0.11698), ('thanks', 0.10025)]
Topic 8: [('does', 0.39692), ('know', 0.25192), ('chip', 0.22492), ('like', 0.17824), ('card', 0.15695)]
Topic 9: [('like', 0.42065), ('card', 0.32249), ('sale', 0.20267), ('video', 0.1571), ('offer', 0.14119)]
Topic 10: [('like', 0.61166), ('